In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

##Abalone DataSet

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/ta-verma/DataSets/master/abalone.csv')
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [0]:
sex = pd.get_dummies(df["Sex"])
sex.head(5)

df = pd.concat([df,sex], axis=1)
df = df.drop('Sex',axis=1)
df.head()

# X = df.drop('Rings',axis=1)
Y = df['Rings']

In [7]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.values)
scaled_features_df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
scaled_features_df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,F,I,M
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,1.571544,-0.674834,-0.688018,1.316677
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,-0.910013,-0.674834,-0.688018,1.316677
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,-0.289624,1.481846,-0.688018,-0.759488
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,0.020571,-0.674834,-0.688018,1.316677
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,-0.910013,-0.674834,1.453451,-0.759488


In [0]:
X = scaled_features_df.drop('Rings',axis=1)

Random Sampling

In [0]:
#Random Sampling

def SimpleClassification(X,Y,ts):
  #Naive-Bayes
  
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=1)

  gnb = GaussianNB()
  gnb.fit(X_train, Y_train)

  Y_pred = gnb.predict(X_test)

  t = PrettyTable(['Alogrithm', 'Accuracy'])
  t.add_row(['Naive Bayes ', metrics.accuracy_score(Y_test , Y_pred) * 100])

  #knn
  knn = KNeighborsClassifier(n_neighbors=3) 
  knn.fit(X_train, Y_train) 
    
  Y_pred = knn.predict(X_test) 
  t.add_row(['KNN ', metrics.accuracy_score(Y_test , Y_pred) * 100])

  #Desicion Tree
  classifier = tree.DecisionTreeClassifier()
  classifier.fit(X_train,Y_train)
  predictions=classifier.predict(X_test)
  t.add_row(['Decision Tree',accuracy_score(Y_test,predictions)*100])
  print(t)

In [28]:
print("Test Size : 0.25")
SimpleClassification(X,Y,0.25)
print('\n')
print("Test Size : 0.33")
SimpleClassification(X,Y,0.33)

Test Size : 0.25
+---------------+-------------------+
|   Alogrithm   |      Accuracy     |
+---------------+-------------------+
|  Naive Bayes  | 9.760765550239233 |
|      KNN      | 20.95693779904306 |
| Decision Tree | 19.23444976076555 |
+---------------+-------------------+


Test Size : 0.33
+---------------+--------------------+
|   Alogrithm   |      Accuracy      |
+---------------+--------------------+
|  Naive Bayes  | 10.297316896301668 |
|      KNN      | 20.449601160261057 |
| Decision Tree | 18.85424220449601  |
+---------------+--------------------+


Hold out 

In [0]:
#Hold out 

def SimpleClassificationHold(X,Y,ts):
  #Naive-Bayes
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=False, test_size=ts )

  gnb = GaussianNB()
  gnb.fit(X_train, Y_train)

  Y_pred = gnb.predict(X_test)

  t = PrettyTable(['Alogrithm', 'Accuracy'])
  t.add_row(["Naive bayes ", metrics.accuracy_score(Y_test , Y_pred) * 100])

  #knn
  knn = KNeighborsClassifier(n_neighbors=3) 
  knn.fit(X_train, Y_train) 
    
  Y_pred = knn.predict(X_test) 
    
  t.add_row(["KNN ", metrics.accuracy_score(Y_test, Y_pred)*100]) 

  #Desicion Tree
  classifier = tree.DecisionTreeClassifier()
  classifier.fit(X_train,Y_train)
  predictions=classifier.predict(X_test)

  t.add_row(['Decision Tree',accuracy_score(Y_test,predictions)*100])
  print(t)

In [30]:
print("Test Size : 0.25")
SimpleClassificationHold(X,Y,0.25)
print('\n')
print("Test Size : 0.33")
SimpleClassificationHold(X,Y,0.33)
  

Test Size : 0.25
+---------------+--------------------+
|   Alogrithm   |      Accuracy      |
+---------------+--------------------+
|  Naive bayes  | 8.899521531100477  |
|      KNN      | 21.435406698564595 |
| Decision Tree | 20.574162679425836 |
+---------------+--------------------+


Test Size : 0.33
+---------------+--------------------+
|   Alogrithm   |      Accuracy      |
+---------------+--------------------+
|  Naive bayes  | 9.862218999274837  |
|      KNN      | 21.537345902828136 |
| Decision Tree | 19.216823785351707 |
+---------------+--------------------+


In [0]:
import warnings
warnings.filterwarnings('ignore')

Cross Validation

In [0]:
#Cross Validation

def ClassifierCrossValid(X,Y):

  classifier = tree.DecisionTreeClassifier()
  gnb = GaussianNB()
  knn = KNeighborsClassifier(n_neighbors=3) 
  treeScore = cross_val_score(classifier, X, Y, cv=4)
  knnScore = cross_val_score(knn, X, Y, cv=4)
  BayesScore = cross_val_score(gnb, X, Y, cv=4)
  t = PrettyTable(['Alogrithm', 'Score'])
  t.add_row(['Naive Bayes ', BayesScore.mean()*100])
  t.add_row(['Decision Tree', treeScore.mean()*100])
  t.add_row(['KNN ' , knnScore.mean()*100])
  print(t)

In [15]:
ClassifierCrossValid(X,Y)

+---------------+--------------------+
|   Alogrithm   |       Score        |
+---------------+--------------------+
|  Naive Bayes  | 10.390864177161818 |
| Decision Tree | 18.88955342902711  |
|      KNN      | 20.44602100863444  |
+---------------+--------------------+


##Breast Cancer DataSet

In [0]:
from sklearn.datasets import load_breast_cancer
Bdf = load_breast_cancer()

X1 = Bdf.data
Y1 = Bdf.target

In [0]:
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(Bdf.data)
# scaled_features_Bdf = pd.DataFrame(scaled_features, index=Bdf.index, columns=Bdf.columns)
# scaled_features_Bdf.head()
# X1 = scaled_features_Bdf

X_std = (X1 - X1.min(axis=0)) / (X1.max(axis=0) - X1.min(axis=0))
X_scaled = X_std * (X1.max(axis=0) - X1.min(axis=0)) + X1.min(axis=0)
X1 = X_scaled

Random Sampling

In [31]:
#Random Sampling
print("Test Size : 0.25")
SimpleClassification(X1,Y1,0.25)
print('\n')
print("Test Size : 0.33")
SimpleClassification(X1,Y1,0.33)

Test Size : 0.25
+---------------+------------------+
|   Alogrithm   |     Accuracy     |
+---------------+------------------+
|  Naive Bayes  | 94.4055944055944 |
|      KNN      | 92.3076923076923 |
| Decision Tree | 92.3076923076923 |
+---------------+------------------+


Test Size : 0.33
+---------------+-------------------+
|   Alogrithm   |      Accuracy     |
+---------------+-------------------+
|  Naive Bayes  | 94.68085106382979 |
|      KNN      |  92.5531914893617 |
| Decision Tree |  95.2127659574468 |
+---------------+-------------------+


Hold out

In [32]:
#Hold out
print("Test Size : 0.25")
SimpleClassificationHold(X1,Y1,0.25)
print('\n')
print("Test Size : 0.33")
SimpleClassificationHold(X1,Y1,0.33)


Test Size : 0.25
+---------------+-------------------+
|   Alogrithm   |      Accuracy     |
+---------------+-------------------+
|  Naive bayes  |  96.5034965034965 |
|      KNN      | 93.00699300699301 |
| Decision Tree | 88.81118881118881 |
+---------------+-------------------+


Test Size : 0.33
+---------------+-------------------+
|   Alogrithm   |      Accuracy     |
+---------------+-------------------+
|  Naive bayes  | 96.27659574468085 |
|      KNN      | 91.48936170212765 |
| Decision Tree |  87.7659574468085 |
+---------------+-------------------+


Cross Validation

In [33]:
# Cross Validation
ClassifierCrossValid(X1,Y1)

+---------------+-------------------+
|   Alogrithm   |       Score       |
+---------------+-------------------+
|  Naive Bayes  | 94.20491480350636 |
| Decision Tree | 92.44681374963065 |
|      KNN      |  92.6253324140648 |
+---------------+-------------------+
